# Gallery Example: Hyper/Erl/1 Reentrant Queue

This example demonstrates a HyperExp/Erlang/1 queueing system with reentrant routing:
- **Arrivals**: Hyperexponential distribution (high variability)
- **Service**: Erlang distribution (low variability)
- **Servers**: 1 server
- **Scheduling**: FCFS
- **Reentrant**: Jobs switch class and pass through the queue again

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_hyperl1_reentrant(): Network {
    """Create Hyper/Erl/1-Reentrant queueing model"""
    val model = Network("Hyper/Erl/1-Reentrant")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model, "Sink")
// Block 2: classes
    val oclass1 = OpenClass(model, "Class1")
    val oclass2 = OpenClass(model, "Class2")
// HyperExp arrival (SCV=64) for Class1, Class2 is disabled at source
    source.setArrival(oclass1, HyperExp.fit_mean_and_scv(1, 64))
    source.setArrival(oclass2, Disabled())
// Erlang service for Class1, Exponential for Class2
    queue.setService(oclass1, Erlang.fit_mean_and_order(0.5, 5))
    queue.setService(oclass2, Exp(3))
// Block 3: topology with class switching (reentrant)
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addClassSwitch(oclass1, oclass2, queue, queue, 1.0)
    P.addRoute(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model
// Create the model
val model = gallery_hyperl1_reentrant()

## About This Model

This model combines:
- **High arrival variability** (HyperExp with SCV=64): Very bursty arrivals
- **Low service variability** (Erlang with SCV=0.2): Regular service times
- **Reentrant routing**: Jobs switch class and pass through again

The high SCV arrivals create significant queue buildup during bursts, while the regular Erlang service provides steady processing.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)